In [1]:
import os
import numpy as np
from qiskit_aer.primitives import Estimator
from qiskit.quantum_info import Statevector, random_clifford

In [2]:
from tomography import SelfGuidedTomography, Mean_Direct_Fidelity, NearSparseTomography

### Initial Parameters

In [3]:
NQs = [2, 3, 4, 5] #, 6]
L = len(NQs)
N = 10
Niter = 10
shots = 1000

simulator_ideal = Estimator(
    backend_options={
        "shots": shots,
        "method": "stabilizer",
    },
    transpile_options={"optimization_level": 0},
)

In [4]:
def simulate(NQ, simulator=simulator_ideal):
    d = 2 ** NQ

    Omega = random_clifford(NQ).to_circuit()
    OmegaM = np.outer(
        np.array(Statevector(Omega)), (np.array(Statevector(Omega))).conj()
    )

    psi0 = np.random.rand(d) + 1j * np.random.rand(d)
    psi0 = psi0 / np.linalg.norm(psi0)
    guess = psi0

    MDF = Mean_Direct_Fidelity(NQ)

    stop_measuring = lambda x: (np.linalg.norm(list(x.values())) > 0.98)

    I_ex = lambda x: 1 - MDF.MeanFidelity(
        1,
        NQ**2,
        x,
        Omega,
        simulator,
        truncation=None,
        stop_measuring=stop_measuring,
    )

    I_th = lambda x: 1 - np.vdot(x, OmegaM @ x) / (np.linalg.norm(x)) ** 2

    Fidelities = []
    Measures = []
    Last = []

    def callback(i, x):
        Last.append(x)
        Fidelities.append(np.real(I_th(x)))
        Measures.append(len(MDF.Measures))
        return None

    postprocessing = lambda x: NearSparseTomography(x, MDF=MDF)

    SelfGuidedTomography(
        I_ex,
        guess,
        num_iter=Niter,
        callback=callback,
        postprocessing=postprocessing,
    )

    Results = [
        np.array(Fidelities),
        np.array(Measures),
        np.array(Last),
    ]

    return Results  # j index the average over the Hilbert space

In [5]:
from joblib import Parallel, delayed

iterator = np.repeat(NQs, N).flatten()

R = Parallel(n_jobs=-1, verbose=11)(delayed(simulate)(int(NQ)) for NQ in iterator)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

In [6]:
# Recover shape [NQ, N]
#R = [[R[i + j * N] for i in range(N)] for j in range(len(NQs))]

In [7]:
Fids, Meas, _ = [[[R[i + j * N][r] for i in range(N)] for j in range(len(NQs))]
                   for r in range(3)]

Fids = np.real(Fids)
Meas = np.array(Meas)

In [8]:
# Save data. Ax order Fids and Meas are: [NQ, Nrun, Niter]

filename = "sgqt_with_postprocessing.npz"

data = {
    "N": [N],
    "shots": [shots],
    "Niter": [Niter],
    "NQs": NQs,
    "Fids": Fids,
    "Meas": Meas,
    "info": ["N, shots, and Niter are single integers in one array each.",
             "NQs is the array of number of qubits.",
             "Fids and Meas have shapes (len(NQs), N, Niter)"],
}

# Make directory "data" if not present
os.makedirs("data", exist_ok=True)

# Save
filepath = os.path.join("data", filename)
np.savez(filepath, **data)